In [3]:
import numpy as np
from numpy import exp,array,random,dot
import pandas as pd

In [4]:
def normalize(matrix):
    n = matrix.shape[1]
    norm_matrix = matrix

    for i in range(n):
        col_mean = np.mean(matrix[:,i])
        col_stdv = np.std(matrix[:,i])
    norm_matrix[:,i] = (norm_matrix[:,i] - col_mean) / col_stdv
    return norm_matrix

def normalize1(matrix):
    return (matrix - matrix.mean()) / matrix.std()

In [5]:
# Reading data from csv
data = pd.read_csv("data/train.csv")

In [6]:
# Classifier data entries to dummy coloumns
#data = pd.get_dummies(data)
#data.to_csv("data/data_afterdum.csv", sep=',',header=True,index=False)

In [26]:
# Spliting the data into Training data and Validation data
train, validate = np.split(data.sample(frac=1), [int(.7*len(data))])
print(len(train),len(validate))
#print(train)
#print(validate)

27281 11692


In [28]:
trn_ids = train[['id']]
#print(train_ids)
trn_y = train[['salary']]
#print(train_y)
trn_features = train.drop('id', axis=1)
trn_features = trn_features.drop('salary', axis=1)
trn_features = trn_features.drop('race', axis=1)
trn_features = trn_features.drop('native-country', axis=1)
trn_features = trn_features.drop('education', axis=1)#correlated
trn_features = pd.get_dummies(trn_features)
trn_features = normalize1(trn_features)
print(trn_features.shape)

(27281, 45)


In [27]:
vld_ids = validate[['id']]
#print(valid_ids)
vld_y = validate[['salary']]
#print(train_y)
vld_features = validate.drop('id', axis=1)
vld_features = vld_features.drop('salary', axis=1)
vld_features = vld_features.drop('native-country', axis=1)
vld_features = vld_features.drop('race', axis=1)
vld_features = vld_features.drop('education', axis=1)
vld_features = pd.get_dummies(vld_features)
vld_features = normalize1(vld_features)
print(vld_features.shape)

(11692, 45)


In [29]:
test_data = pd.read_csv("data/test.csv")
test_ids = test_data['id']
#print(test_ids)
tst_features = test_data.drop('id', axis=1)
tst_features = tst_features.drop('native-country', axis=1)
tst_features = tst_features.drop('race', axis=1)
tst_features = tst_features.drop('education', axis=1)
tst_features = pd.get_dummies(tst_features)
tst_features = normalize1(tst_features)
print(tst_features.shape)

(6878, 45)


In [11]:
# Now Neural Network implementation starts

# Activation functions
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x))

def derv_sigmoid(x):
    return x * (1 - x)

def tanh(x):
        return np.tanh(x)

def derv_tanh(x):
        return (1.0 - np.square(np.tanh(x)))


In [12]:
class NeuralNetwork():
    def __init__(self):
        
        # Number of nodes in each Layer
        l1_nodes=45
        l2_nodes=30
        l3_nodes=30
        l4_nodes=1

        # Assigning random weights to Neural Network
        self.weights1 = 2 * random.random((l1_nodes, l2_nodes)) -1
        self.weights2 = 2 * random.random((l2_nodes, l3_nodes)) -1
        self.weights3 = 2 * random.random((l3_nodes, l4_nodes)) -1

    # Activation functions
    def tanh(self,x):
        return np.tanh(x)

    def derv_tanh(self,x):
        return (1.0 - (np.tanh(x))**2)

    # Train neural network using backpropogation
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations,eta):
        for i in range(number_of_training_iterations):
            # print(i)
            
            # Forward Pass
            a2 = self.tanh(dot(training_set_inputs, self.weights1))
            a3 = self.tanh(dot(a2, self.weights2))
            output = self.tanh(dot(a3, self.weights3))

            # Backpopogation Pass
            # Error for each layer
            del4 = (training_set_outputs - output)*self.derv_tanh(output)
            print(i,np.mean(np.square(del4)))
            del3 = dot(self.weights3, del4.T)*(self.derv_tanh(a3).T)
            del2 = dot(self.weights2, del3)*(self.derv_tanh(a2).T)

            # Adjustments (gradients) for each layer
            adjustment3 = eta*dot(a3.T, del4)
            adjustment2 = eta*dot(a2.T, del3.T)
            adjustment1 = eta*dot(training_set_inputs.T, del2.T)

            # Adjusting weights accordingly
            self.weights1 += adjustment1
            self.weights2 += adjustment2
            self.weights3 += adjustment3

        return self.weights1,self.weights2,self.weights3


In [30]:
class NeuralNetwork1():
    def __init__(self):
        
        # Number of nodes in each Layer
        l1_nodes=45
        l2_nodes=3
        l3_nodes=1

        # Assigning random weights to Neural Network
        self.weights1 = 2 * random.random((l1_nodes, l2_nodes)) -1
        self.weights2 = 2 * random.random((l2_nodes, l3_nodes)) -1

    # Activation functions
    def tanh(self,x):
        return np.tanh(x)

    def derv_tanh(self,x):
        return (1.0 - (np.tanh(x))**2)

    # Train neural network using backpropogation
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations,eta):
        for i in range(number_of_training_iterations):
            # print(i)
            
            # Forward Pass
            a2 = self.tanh(dot(training_set_inputs, self.weights1))
            output = self.tanh(dot(a2, self.weights2))

            # Backpopogation Pass
            # Error for each layer
            del3 = (training_set_outputs - output)*self.derv_tanh(output)
            print(i,np.mean(np.square(del3)))
            del2 = dot(self.weights2, del3.T)*(self.derv_tanh(a2).T)

            # Adjustments (gradients) for each layer
            adjustment2 = eta*dot(a2.T, del3)
            adjustment1 = eta*dot(training_set_inputs.T, del2.T)

            # Adjusting weights accordingly
            self.weights1 += adjustment1
            self.weights2 += adjustment2

        return self.weights1,self.weights2

In [32]:
# Initialising Neural Network
neural_network = NeuralNetwork1()

#print ("Starting weights Layer 1: ")
#print (neural_network.weights1)
#print ("\nStarting weights Layer 2: ")
#print (neural_network.weights2)
#print ("\nStarting weights Layer 3: ")
#print (neural_network.weights3)

# Training Set
training_set_inputs = np.array(trn_features)
training_set_outputs = np.array(trn_y.T)

#fweights1,fweights2,fweights3=neural_network.train(training_set_inputs, training_set_outputs.T,100,0.00003)
fweights1,fweights2=neural_network.train(training_set_inputs, training_set_outputs.T,1000,0.00001)

#print ("\nNew weights Layer 1 after training:")
#print (neural_network.weights1)
#print ("\nNew weights Layer 2 after training: ")
#print (neural_network.weights2)
#print ("\nNew weights Layer 3 after training: ")
#print (neural_network.weights3)

# Validation
after_l1 = tanh(np.dot(vld_features,fweights1))
after_l2 = tanh(np.dot(after_l1,fweights2))
#print("after_l3")
#print(after_l3)
predicted = []

after_l2 = after_l2.ravel()
#print(after_l3)

#Threshholding values
for i in range(len(after_l2)):
    if after_l2[i] > 0:
        predicted.append([1])
    else:
        predicted.append([0])

#Validating accuracy of Neural network results
vld_y = np.array(vld_y).ravel()
#print(vld_y)
#np.savetxt("data/vld_y.csv",vld_y,delimiter=',',fmt="%d",header="id,salary",comments ='')
c=0
for i in range(len(predicted)):
    if(predicted[i] == vld_y[i]):
        c+=1
vld_acur=c/len(predicted)
print(vld_acur)


0 0.236981825864
1 0.237508722084
2 0.231891537542
3 0.220231123033
4 0.214493937166
5 0.213385869198
6 0.212570088948
7 0.211800423432
8 0.211115704478
9 0.210489604867
10 0.209895641298
11 0.209318593777
12 0.208750978347
13 0.208189356182
14 0.207632220975
15 0.207078953074
16 0.206529328348
17 0.20598329491
18 0.205440876495
19 0.204902134043
20 0.204367151322
21 0.203836027592
22 0.203308870093
23 0.202785785124
24 0.20226686975
25 0.201752207208
26 0.201241868182
27 0.200735918138
28 0.200234428825
29 0.199737490506
30 0.199245221171
31 0.198757769691
32 0.198275311481
33 0.19779803723
34 0.197326137008
35 0.196859782952
36 0.196399113704
37 0.19594422306
38 0.195495154235
39 0.195051900001
40 0.194614407957
41 0.194182589371
42 0.193756329716
43 0.193335499116
44 0.192919961387
45 0.192509581036
46 0.192104228095
47 0.191703781027
48 0.191308127927
49 0.190917166199
50 0.190530800854
51 0.190148941718
52 0.18977150014
53 0.189398385951
54 0.189029505475
55 0.188664761065
56 0.18

In [ ]:
# Getting final results from test data
after_l1 = tanh(np.dot(tst_features,fweights1))
after_l2 = tanh(np.dot(after_l1,fweights2))
after_l3 = tanh(np.dot(after_l2,fweights3))
print("after_l3")
print(after_l3)
predicted = []
after_l3 = after_l3.ravel()

result=np.column_stack((test_ids,after_l3))
np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')
print(after_l3)
for i in range(len(after_l3)):
    if after_l3[i] > 0:
        predicted.append(1)
    else:
        predicted.append(0)
print("predicted")
print(predicted)
result=np.column_stack((test_ids,(after_l3)))

np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')

In [34]:
# Getting final results from test data
after_l1 = tanh(np.dot(tst_features,fweights1))
after_l2 = tanh(np.dot(after_l1,fweights2))
print("after_l2")
print(after_l2)
predicted = []
after_l2 = after_l2.ravel()

result=np.column_stack((test_ids,after_l2))
np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')
print(after_l2)
for i in range(len(after_l2)):
    if after_l2[i] > 0:
        predicted.append(1)
    else:
        predicted.append(0)
print("predicted")
print(predicted)
result=np.column_stack((test_ids,(after_l2)))

np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')

after_l2
[[-0.13650391]
 [-0.318587  ]
 [ 0.33493801]
 ..., 
 [-0.23909666]
 [-0.25220552]
 [ 0.25789659]]
[-0.13650391 -0.318587    0.33493801 ..., -0.23909666 -0.25220552
  0.25789659]
predicted
[0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0